### Algorytmy Macierzowe lab 2

#### Konfiguracja

In [1]:
import numpy as np
import scipy.linalg
from math import log, ceil, pow
from typing import Tuple, List

#### Generowanie macierzy

In [2]:
def generate_matrix(ncols: int, nrows: int):
    return np.random.uniform(low=0.00000001, high=1.0, size=(nrows, ncols))

#### Licznik operacji

In [33]:
class Counter:
    def __init__(self) -> None:
        self.add_counter = 0
        self.mul_counter = 0
        self.div_counter = 0
        self.sub_counter = 0
        self.operation_counter = 0

    def add(self, A: np.ndarray, B: np.ndarray) -> np.ndarray:
        self.add_counter += A.size
        self.operation_counter += A.size
        return A + B

    def mul(self, A: np.ndarray, B: np.ndarray) -> np.ndarray:
        self.mul_counter += 1
        self.operation_counter += int(
            (A.shape[0]) ** 2.807
        )  # we assume Strassen algorithm for multiplication
        return A @ B

    def inc_flops_mul(self, count: int):
        self.operation_counter += count

    def print_counters(self) -> None:
        print(f"Number of add operations: {self.add_counter}")
        print(f"Number of sub operations: {self.sub_counter}")
        print(f"Number of div operations: {self.div_counter}")
        print(f"Number of mul operations: {self.mul_counter}")
        print(f"Number of unit operations: {self.operation_counter}")

#### Funckje pomocniczne

In [4]:
def resize_matrix_to_2n(A: np.ndarray):
    """Change size of matrix to simplify processing"""
    size_A = A.shape
    new_height: int = 0
    new_width: int = 0

    new_height = find_next_power_of_2(size_A[0])

    new_width = find_next_power_of_2(size_A[1])

    new_A = np.pad(
        A, [(0, new_height - size_A[0]), (0, new_width - size_A[1])], mode="constant"
    )

    return new_A


def split(array: np.ndarray, n: int) -> Tuple:
    """Split a matrix into sub-matrices"""
    return (
        array[:n, :n],
        array[:n, n:],
        array[n:, :n],
        array[n:, n:],
    )


def find_next_power_of_2(number: int) -> int:
    """Finds closest number that is power of 2"""
    return int(pow(2, ceil(log(number) / log(2))))

#### Rekurencyjne odwracanie macierzy

In [30]:
A = generate_matrix(5, 5)


def recursive_matrix_inverse(A: np.ndarray, counter: Counter) -> np.ndarray:
    add = counter.add

    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0] // 2
        A11, A12, A21, A22 = split(A, split_at)

        A11_inv = recursive_matrix_inverse(A11, counter)
        S22 = add(A22, -mul(mul(A21, A11_inv), A12))
        S22_inv = recursive_matrix_inverse(S22, counter)

        B11 = mul(
            A11_inv,
            add(np.identity(A11.shape[0]), mul(mul(mul(A12, S22_inv), A21), A11_inv)),
        )
        B12 = -mul(mul(A11_inv, A12), S22_inv)
        B21 = -mul(mul(S22_inv, A21), A11_inv)
        B22 = S22_inv

        return np.concatenate(
            [np.concatenate([B11, B12], axis=1), np.concatenate([B21, B22], axis=1)],
            axis=0,
        )
    else:
        return np.full((1, 1), 1 / A[0][0])

counter = Counter()
print(recursive_matrix_inverse(A.copy(), counter))
counter.print_counters()
print()
print(np.linalg.inv(A))

[[ 0.45021678 -0.55801318  1.88691342 -0.72076973 -0.68208376]
 [-4.24033418 -1.61011382  1.97775614  0.55812027  3.98582818]
 [-0.27105761  0.6451497  -2.26840136 -0.32757359  2.55782594]
 [ 2.9447462   0.64428868 -1.1809448   1.10728012 -2.84373238]
 [-2.40986406  0.99443373 -0.22316482 -0.71108446  2.34737701]]
Number of add operations: 22
Number of sub operations: 0
Number of div operations: 0
Number of mul operations: 96
Number of unit operations: 1662

[[ 0.45021678 -0.55801318  1.88691342 -0.72076973 -0.68208376]
 [-4.24033418 -1.61011382  1.97775614  0.55812027  3.98582818]
 [-0.27105761  0.6451497  -2.26840136 -0.32757359  2.55782594]
 [ 2.9447462   0.64428868 -1.1809448   1.10728012 -2.84373238]
 [-2.40986406  0.99443373 -0.22316482 -0.71108446  2.34737701]]


#### Rekurencyjna LU faktoryzacja

In [31]:
np.random.seed(42)
A = generate_matrix(5, 5)


def recursive_lu(A: np.ndarray, counter: Counter) -> Tuple[np.ndarray, np.ndarray]:
    add = counter.add
    mul = counter.mul
    if A.size > 1:
        split_at = A.shape[0] // 2
        A11, A12, A21, A22 = split(A, split_at)

        L11, U11 = recursive_lu(A11, counter)
        U11_inv = recursive_matrix_inverse(U11, counter)

        L21 = mul(A21, U11_inv)
        L11_inv = recursive_matrix_inverse(L11, counter)

        U12 = mul(L11_inv, A12)
        L22 = add(A22, -mul(mul(mul(A21, U11_inv), L11_inv), A12))

        Ls, Us = recursive_lu(L22, counter)
        U22 = Us
        L22 = Ls

        return np.concatenate(
            [
                np.concatenate([L11, np.full((L21.shape[1], L21.shape[0]), 0)], axis=1),
                np.concatenate([L21, L22], axis=1),
            ],
            axis=0,
        ), np.concatenate(
            [
                np.concatenate([U11, U12], axis=1),
                np.concatenate([np.full((U12.shape[1], U12.shape[0]), 0), U22], axis=1),
            ],
            axis=0,
        )
    else:
        return np.full((1, 1), 1), np.full((1, 1), A[0, 0])


counter = Counter()
print(f"Original matrix A:\n{A}\n")
L, U = recursive_lu(A, counter)
print(f"L:\n{L}\nU:\n{U}\n\nL*U=\n{L @ U}\n\n")
counter.print_counters()
print()
L, U = scipy.linalg.lu(A, permute_l=True)
print(f"Scipy L*U=\n{L @ U}")

Original matrix A:
[[0.37454013 0.95071431 0.73199394 0.59865849 0.15601865]
 [0.15599453 0.05808362 0.86617615 0.60111502 0.70807258]
 [0.0205845  0.96990985 0.83244264 0.21233912 0.18182498]
 [0.18340452 0.30424225 0.52475644 0.43194502 0.29122915]
 [0.6118529  0.13949387 0.29214466 0.36636185 0.45606999]]

L:
[[  1.           0.           0.           0.           0.        ]
 [  0.41649617   1.           0.           0.           0.        ]
 [  0.05495941  -2.71588996   1.           0.           0.        ]
 [  0.48967922   0.47738928  -0.04387629   1.           0.        ]
 [  1.63361108   4.1836794   -1.40373424 -23.74302445   1.        ]]
U:
[[ 0.37454013  0.95071431  0.73199394  0.59865849  0.15601865]
 [ 0.         -0.33788525  0.56130347  0.35177605  0.64309141]
 [ 0.          0.          2.31665115  1.13482223  1.91981579]
 [ 0.          0.          0.          0.02065207 -0.0079405 ]
 [ 0.          0.          0.          0.          0.01708756]]

L*U=
[[0.37454013 0.95071

#### Rekurencyjny wyznacznik macierzy

In [40]:
import functools
np.random.seed(42)
A = generate_matrix(5, 5)


def recursive_determinant(A: np.ndarray, counter: Counter) -> float:
    _, U = recursive_lu(A, counter)

    U_diag = np.diagonal(U)

    det = functools.reduce(lambda x, y: x * y, U_diag)
    counter.inc_flops_mul(U_diag.size - 1) # wykonaliśmy N - 1 mnożeń floatów

    return det

counter = Counter()
print(recursive_determinant(A, counter))
counter.print_counters()
print()
print(np.linalg.det(A))

-0.00010345989814894179
Number of add operations: 19
Number of sub operations: 0
Number of div operations: 0
Number of mul operations: 67
Number of unit operations: 734

-0.00010345989814894385


#### Wykresy

In [ ]:
import matplotlib.pyplot as plt
from time import time
from matplotlib.ticker import FormatStrFormatter


times_binet: List[float] = []
flops_binet: List[int] = []

power_basis = list(range(2, 11))
powers = [2**k for k in power_basis]

for k in power_basis:
    A = np.random.rand(2**k, 2**k)
    B = np.random.rand(2**k, 2**k)

    counter = Counter()
    start_time: float = time()

    binet_algorithm(A, B, counter)

    total_time: float = time() - start_time

    times_binet.append(total_time)
    flops_binet.append(counter.operation_counter)

In [ ]:
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%d s"))
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Czas")
plt.plot(powers, times_binet)

In [ ]:
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Ilość flopsów")
plt.plot(powers, flops_binet)

In [ ]:
times_strassen: List[float] = []
flops_strassen: List[int] = []

for k in power_basis:
    A = np.random.rand(2**k, 2**k)
    B = np.random.rand(2**k, 2**k)

    counter = Counter()
    start_time: float = time()

    strassen_algorith(A, B, counter)

    total_time: float = time() - start_time

    times_strassen.append(total_time)
    flops_strassen.append(counter.operation_counter)

In [ ]:
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%d s"))
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Czas")
plt.plot(powers, times_strassen)

In [ ]:
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Ilość flopsów")
plt.plot(powers, flops_strassen)

In [ ]:
times_winograd: List[float] = []
flops_winograd: List[int] = []

for k in power_basis:
    A = np.random.rand(2**k, 2**k)
    B = np.random.rand(2**k, 2**k)

    counter = Counter()
    start_time: float = time()

    winograd_algorithm(A, B, counter)

    total_time: float = time() - start_time

    times_winograd.append(total_time)
    flops_winograd.append(counter.operation_counter)

In [ ]:
plt.gca().yaxis.set_major_formatter(FormatStrFormatter("%d s"))
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Czas")
plt.plot(powers, times_winograd)

In [ ]:
plt.xlabel("Rozmiar macierzy")
plt.ylabel("Ilość flopsów")
plt.plot(powers, flops_winograd)

In [ ]:
A = np.array([[1, 3, 5], [2, 4, 6], [3, 5, 7]])
B = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

print(winograd_algorithm(A, B, Counter()))